In [46]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

1

In [29]:
# 加载 ratings 数据
ratings = pd.read_csv(r'C:\Users\lenovo\Desktop\学到发疯\数据挖掘作业\ml-100k\u.data', 
                      sep='\t', 
                      names=['user_id', 'item_id', 'rating', 'timestamp'])

In [30]:
#创建用户物品评分矩阵
utility_matrix=ratings.pivot_table(index='user_id',columns='item_id',values='rating')

In [31]:
#平均评分
user_means = utility_matrix.mean(axis=1)

In [32]:
#中心化评分（每个评分减去该用户的平均值）
centered_matrix = utility_matrix.sub(user_means, axis=0)

In [33]:
# === 寻找与用户1最相似的10个用户 ===
#从用户-物品中心化矩阵中取出用户1的评分向量。
#相似度计算后将保存在 similarities 列表中，格式为 (用户编号, 相似度)。
user1_id=1
user1=centered_matrix.loc[user1_id] # 获取用户1的中心化评分向量
similarities = []  # 用于保存所有用户与用户1的相似度

In [34]:
#对每个其他用户计算与用户1的相似度：
for user in centered_matrix.index:
    if user == user1_id:
        continue  # 跳过用户1自己

    user_centered = centered_matrix.loc[user]  # 获取当前用户的中心化评分向量
    common_items = user1.notna() & user_centered.notna()  # 两个用户都评分的电影
#计算余弦相似度
    if common_items.any():
        u1 = user1[common_items].values.astype(float)
        u = user_centered[common_items].values.astype(float)
        dot_product = np.dot(u1, u)
        norm_u1 = np.linalg.norm(u1)
        norm_u = np.linalg.norm(u)
        cosine = dot_product / (norm_u1 * norm_u) if norm_u1 * norm_u != 0 else 0.0
    else:
        cosine = 0.0
    similarities.append((user, cosine))

In [35]:
#获取最相似的10个用户：
top10_users = sorted(similarities, key=lambda x: x[1], reverse=True)[:10]
print(f"\nThe 10 most similar users to user 1 are: {[u for u, _ in top10_users]}")



The 10 most similar users to user 1 are: [418, 155, 341, 685, 812, 351, 811, 166, 810, 309]


In [36]:
# === 预测用户1对电影508的评分 ===
item_id = 508
numerator = 0.0
denominator = 0.0

for (user, sim) in top10_users:
    if np.isnan(utility_matrix.loc[user, item_id]):  # 跳过未评分的用户
        continue
    rating_centered = centered_matrix.loc[user, item_id]
    numerator += sim * rating_centered
    denominator += sim

predicted_centered = numerator / denominator if denominator != 0 else 0
predicted_rating = user_means[user1_id] + predicted_centered

print(f"\nThe predicted rating for user {user1_id} on item {item_id} is: {predicted_rating:.2f}")


The predicted rating for user 1 on item 508 is: 3.61


2